In [7]:
import tensorflow as tf
import pandas as pd
import keras_tuner as kt
# from sklearn.model_selection import train_test_split

tf.__version__

'2.14.0'

In [8]:
# Using same columns as with Ex 6.01 for features
USECOLS = ["AAGE", "ADTIND", "ADTOCC", "SEOTR", "WKSWORK", "PTOTVAL"]

df = pd.read_csv("../input_data/CSVs/census-income-train.csv", usecols=USECOLS)
target = df.pop("PTOTVAL")
df.head()

,AAGE,ADTIND,ADTOCC,SEOTR,WKSWORK
0,73,0,0,0,0
1,58,4,34,0,52
2,18,0,0,0,0
3,9,0,0,0,0
4,10,0,0,0,0


In [9]:
# Split data into test and train sets
features_train = df.iloc[:15000]
target_train = target.iloc[:15000]

features_test = df.iloc[15000:]
target_test = target.iloc[15000:]
target_test.head()

15000     896.10
15001    2349.06
15002    1134.00
15003    2042.50
15004    4521.29
Name: PTOTVAL, dtype: float64

In [10]:
tf.random.set_seed(8)
# `hp` = hyperparameters
def model_builder(hp):
    model = tf.keras.models.Sequential()
    
    hp_l2 = hp.Choice("l2", values=[0.1, 0.01, 0.001])
    hp_units = hp.Int('units', min_value=128, max_value=512, step=64)
    
    in_layer = tf.keras.layers.Dense(hp_units, input_shape=(5,), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    hidden_layer_1 = tf.keras.layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    hidden_layer_2 = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    hidden_layer_3 = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l=hp_l2))
    out_layer = tf.keras.layers.Dense(1)
    
    model.add(in_layer)
    model.add(hidden_layer_1)
    model.add(hidden_layer_2)
    model.add(hidden_layer_3)
    model.add(out_layer)
    
    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001])
    optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model

In [11]:
# Tune the model probabilistically, using mean squared error as the metric to optimise the model
tuner = kt.tuners.BayesianOptimization(model_builder, objective="val_mse", max_trials=10, project_name="income_predictor_bayesian")
# Search for the best hyperparameters
tuner.search(features_train, target_train, validation_data=(features_test, target_test))
best_hps = tuner.get_best_hyperparameters()[0]

Trial 10 Complete [00h 00m 20s]
val_mse: 1047347.75

Best val_mse So Far: 1035517.3125
Total elapsed time: 00h 03m 09s


In [13]:
best_l2 = best_hps.get("l2")
best_l2

0.001

In [14]:
best_input_units = best_hps.get("units")
best_input_units

256

In [15]:
best_lr = best_hps.get("learning_rate")
best_lr

0.001

In [16]:
# Build and train the model based on the optimal hyperparameters found
model = tuner.hypermodel.build(best_hps)
model.fit(features_train, target_train, epochs=5, validation_data=(features_test, target_test))

Epoch 1/5
469/469 [==============================] - 17s 33ms/step - loss: 1466548.2500 - mse: 1466546.8750 - val_loss: 1044222.1250 - val_mse: 1044221.1250
Epoch 2/5
469/469 [==============================] - 16s 34ms/step - loss: 1012640.7500 - mse: 1012639.9375 - val_loss: 1016550.9375 - val_mse: 1016549.5000
Epoch 3/5
469/469 [==============================] - 14s 29ms/step - loss: 1007892.1250 - mse: 1007890.9375 - val_loss: 1032201.4375 - val_mse: 1032200.2500
Epoch 4/5
469/469 [==============================] - 14s 30ms/step - loss: 1002795.3750 - mse: 1002794.1250 - val_loss: 1019208.6250 - val_mse: 1019207.1250
Epoch 5/5
469/469 [==============================] - 14s 30ms/step - loss: 996341.4375 - mse: 996339.8750 - val_loss: 1013901.3750 - val_mse: 1013900.3125
